In [1]:
import re
import requests

from bs4 import BeautifulSoup
from collections import namedtuple

In [2]:
url_pattern = re.compile(r'''\b((ftp|https?)://[-\w]+(\.\w[-\w]*)+|(?i:[a-z0-9](?:[-a-z0-9]*[a-z0-9])?\.)+(?-i:com\b|edu\b|biz\b|gov\b|in(?:t|fo)\b|mil\b|net\b|org\b|[a-z][a-z]\b))(:\d+)?(/[^.!,?;"'<>()\[\]{}\s\x7F-\xFF]*(?:[.!,?]+[^.!,?;"'<>()\[\]{}\s\x7F-\xFF]+)*)?''')
a = url_pattern.match('http://harttle.com/2016/02/23/javascript-regular-expressions.html')

In [3]:
class movie:
    def __init__(self, params):
        assert isinstance(params, dict)
        self.cover = params.get('cover')
        self.id = params.get('id')
        self.is_new = params.get('is_new')
        self.playable = params.get('playable')
        self.rate = params.get('rate')
        self.title = params.get('title')
        self.url = params.get('url')
    @property
    def show(self):
        print('title : %s' % (self.title))
        print('rate : %s' % (self.rate))
        print('id : %s' % (self.id))
    def check_url(self):
        url_pattern = re.compile(r'''\b((ftp|https?)://[-\w]+(\.\w[-\w]*)+|(?i:[a-z0-9](?:[-a-z0-9]*[a-z0-9])?\.)+(?-i:com\b|edu\b|biz\b|gov\b|in(?:t|fo)\b|mil\b|net\b|org\b|[a-z][a-z]\b))(:\d+)?(/[^.!,?;"'<>()\[\]{}\s\x7F-\xFF]*(?:[.!,?]+[^.!,?;"'<>()\[\]{}\s\x7F-\xFF]+)*)?''')
        return url_pattern.match(self.url)
    def get_detail(self):
        assert self.check_url(), 'Wrong url format : {}'.format(self.url)
        headers = {'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:54.0) Gecko/20100101 Firefox/54.0'}
        response = requests.get(self.url, headers=headers)
        self.detail = response.text
    @property
    def show_detail(self):
        print(self.detail)
    def extract_movie(self):
        home_url = 'https://movie.douban.com'
        movie_pattern = re.compile('/subject/[0-9]+?/')
        movie_index = movie_pattern.findall(self.detail)
        self.movies_url = [home_url + x for x in movie_index]
        return self.movies_url


In [4]:
url = 'https://movie.douban.com/j/search_subjects?type=movie&tag=最新&page_limit=1&page_start=100'

response = requests.get(url, verify=False)
# print(response.json())
a = response.json()['subjects'][0]
a.keys()
# a.get('rate')
# a['url'] = 'fasfd'
a

/home/da/anaconda3/envs/spyder/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'cover': 'https://img1.doubanio.com/view/movie_poster_cover/lpst/public/p2492665487.jpg',
 'cover_x': 705,
 'cover_y': 1000,
 'id': '26270502',
 'is_new': False,
 'playable': True,
 'rate': '7.4',
 'title': '绣春刀II：修罗战场',
 'url': 'https://movie.douban.com/subject/26270502/'}

In [5]:
aa = movie(a)

In [6]:
aa.get_detail()

In [8]:
aa.show_detail()

<!DOCTYPE html>
<html lang="zh-cmn-Hans" class="ua-linux ua-ff54">
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <meta name="renderer" content="webkit">
    <meta name="referrer" content="always">
    <title>
        绣春刀II：修罗战场 (豆瓣)
</title>
    
    <meta name="baidu-site-verification" content="cZdR4xxR7RxmM4zE" />
    <meta http-equiv="Pragma" content="no-cache">
    <meta http-equiv="Expires" content="Sun, 6 Mar 2005 01:00:00 GMT">
    
    <link rel="apple-touch-icon" href="https://img3.doubanio.com/f/movie/d59b2715fdea4968a450ee5f6c95c7d7a2030065/pics/movie/apple-touch-icon.png">
    <link href="https://img3.doubanio.com/f/shire/94213e812acbb00123f685909b4768bb304d16f3/css/douban.css" rel="stylesheet" type="text/css">
    <link href="https://img3.doubanio.com/f/shire/ae3f5a3e3085968370b1fc63afcecb22d3284848/css/separation/_all.css" rel="stylesheet" type="text/css">
    <script type="text/javascript">var _head_start = new Date();</script>
    <s

TypeError: 'NoneType' object is not callable

In [190]:
home_url = 'https://movie.douban.com'
movie_pattern = re.compile('/subject/[0-9]+?/')
movie_index = movie_pattern.findall(aa.detail)
movies = [home_url + x for x in movie_index]
movies

['https://movie.douban.com/subject/26635321/',
 'https://movie.douban.com/subject/26635321/',
 'https://movie.douban.com/subject/26635321/',
 'https://movie.douban.com/subject/26635321/',
 'https://movie.douban.com/subject/26635321/',
 'https://movie.douban.com/subject/26635321/',
 'https://movie.douban.com/subject/26635321/',
 'https://movie.douban.com/subject/26635321/',
 'https://movie.douban.com/subject/26635321/',
 'https://movie.douban.com/subject/26635321/',
 'https://movie.douban.com/subject/4925418/',
 'https://movie.douban.com/subject/4925418/',
 'https://movie.douban.com/subject/26605865/',
 'https://movie.douban.com/subject/26605865/',
 'https://movie.douban.com/subject/24861238/',
 'https://movie.douban.com/subject/24861238/',
 'https://movie.douban.com/subject/25702678/',
 'https://movie.douban.com/subject/25702678/',
 'https://movie.douban.com/subject/10569151/',
 'https://movie.douban.com/subject/10569151/',
 'https://movie.douban.com/subject/24460604/',
 'https://movie

In [11]:
url = 'https://movie.douban.com/subject/26270502/'
headers = {'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:54.0) Gecko/20100101 Firefox/54.0'}
response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, 'lxml')
# articles = soup.find_all('textarea')
# result = articles[0].string

'\n        绣春刀II：修罗战场 (豆瓣)\n'

In [131]:
def wrap_url(url):
    return 'https://movie.douban.com'+url

In [158]:
people_info = soup.find_all('div', attrs={'id':'info'})[0]

# 标题
title = soup.title.string.strip()

# 导演
director_temp = people_info.find_all('a', attrs={'rel':'v:directedBy'})[0]
director_name = director_temp.string
director_url = wrap_url(director_temp['href'])
director = {director_name:director_url}

# 编剧
pattern_composer = re.compile('''<a href="(/celebrity/[0-9]+?/)">(.+?)</a>''')
composer_temp = pattern_composer.findall(str(people_info))
composer = {}
for (x, y) in composer_temp:
    composer[y] = wrap_url(x)

# 主演
actors_temp = people_info.find_all('a', attrs={'rel':'v:starring'})
actors = {}
for actor in actors_temp:
    name = actor.string
    url = actor['href']
    actors[name] = wrap_url(url)
    
# 电影类型

movie_class_temp = people_info.find_all('span', attrs={'property':'v:genre'})
movie_class = [x.string for x in movie_class_temp]

# 制片国家/地区

pattern_place = re.compile('<span class="pl">制片国家/地区:</span>(.+?)<br/>')
place = pattern_place.findall(str(people_info))[0].strip()

# 语言

pattern_language = re.compile('<span class="pl">语言:</span>(.+?)<br/>')
language = pattern_language.findall(str(people_info))[0].strip()

# 上映时间

time = people_info.find_all('span', attrs={'property':'v:initialReleaseDate'})[0].string.strip()

# 片长

length = people_info.find_all('span', attrs={'property':'v:runtime'})[0].string.strip()

# 别名

pattern_alias = re.compile('<span class="pl">又名:</span>(.+?)<br/>')
alias = pattern_alias.findall(str(people_info))[0].strip()

# imdb链接

IMDB_url = people_info.find_all('a', attrs={'rel':'nofollow'})[0]['href']

people_info


<div id="info">
<span><span class="pl">导演</span>: <span class="attrs"><a href="/celebrity/1321200/" rel="v:directedBy">路阳</a></span></span><br/>
<span><span class="pl">编剧</span>: <span class="attrs"><a href="/celebrity/1317594/">陈舒</a> / <a href="/celebrity/1321200/">路阳</a> / <a href="/celebrity/1377577/">禹扬</a></span></span><br/>
<span class="actor"><span class="pl">主演</span>: <span class="attrs"><a href="/celebrity/1077991/" rel="v:starring">张震</a> / <a href="/celebrity/1052359/" rel="v:starring">杨幂</a> / <a href="/celebrity/1274761/" rel="v:starring">张译</a> / <a href="/celebrity/1312940/" rel="v:starring">雷佳音</a> / <a href="/celebrity/1318720/" rel="v:starring">辛芷蕾</a> / <a href="/celebrity/1275482/" rel="v:starring">金士杰</a> / <a href="/celebrity/1376605/" rel="v:starring">刘端端</a> / <a href="/celebrity/1351719/" rel="v:starring">武强</a> / <a href="/celebrity/1342478/" rel="v:starring">杨轶</a> / <a href="/celebrity/1314374/" rel="v:starring">李媛</a> / <a href="/celebrity/1315721/" rel="